<a href="https://colab.research.google.com/github/psst54/startTensorflow/blob/master/weekly/5_1_%EB%8B%A8%EC%96%B4_%EB%8B%A8%EC%9C%84_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/wikibook/tf2/blob/master/Chapter7.ipynb 를 따름

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

국사편찬위원회에서 제공하는 조선왕조실록 데이터를 다운받는다

In [ ]:
path_to_file = tf.keras.utils.get_file('input.txt', 'http://bit.ly/2Mc3SOV')

train_text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print('Length of text: {} characters'.format(len(train_text)))
print()

print(train_text[:100])

62013440/62012502 [==============================] - 1s 0us/step
Length of text: 26265493 characters

﻿태조 이성계 선대의 가계. 목조 이안사가 전주에서 삼척·의주를 거쳐 알동에 정착하다 
태조 강헌 지인 계운 성문 신무 대왕(太祖康獻至仁啓運聖文神武大王)의 성은 이씨(李氏)요, 휘


한글과 개행 문자만 남도록 처리하는 과정이다.

개행문자를 기준으로 train_text를 나누고, 각 문장을 clean_str() 함수의 인자로 넘겨준다. 그리고 각 문장을 띄어쓰기 단위로 자른 다음 개행문자와 함께 train_text_X에 저장한다.

In [ ]:
import re
# From https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
def clean_str(string):    
    string = re.sub(r"[^가-힣A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", "", string)
    string = re.sub(r"\)", "", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"\'{2,}", "\'", string)
    string = re.sub(r"\'", "", string)

    return string
    

train_text = train_text.split('\n')
train_text = [clean_str(sentence) for sentence in train_text]
train_text_X = []

for sentence in train_text:
    train_text_X.extend(sentence.split(' '))
    train_text_X.append('\n')

train_text_X = [word for word in train_text_X if word != '']

print(train_text_X[:20])

['태조', '이성계', '선대의', '가계', '목조', '이안사가', '전주에서', '삼척', '의주를', '거쳐', '알동에', '정착하다', '\n', '태조', '강헌', '지인', '계운', '성문', '신무', '대왕']


vocab에는 train_text_X에 저장된 모든 단어들이 중복 없이 정렬되어 있다. 맨 마지막 원소로 'UNK' 토큰을 넣고 word2idx, idx2word

In [ ]:
vocab = sorted(set(train_text_X))
vocab.append('UNK')
# UNK는 이후 임의의 문장을 입력할 때 텍스트에 미리 준비되어 있지 않은 단어를 사용할 경우에 대한 토큰
print('{} unique words'.format(len(vocab)))

word2idx = {u:i for i, u in enumerate(vocab)}
idx2word = np.array(vocab)

text_as_int = np.array([word2idx[c] for c in train_text_X])

print('{')
for word,_ in zip(word2idx, range(10)):
    print(' {:4s}: {:3d},'.format(repr(word), word2idx[word]))
print(' ...\n}');

print('index of UNK: {}'.format(word2idx['UNK']))

332640 unique words
{
 '\n':   0,
 '!' :   1,
 ',' :   2,
 '000명으로':   3,
 '001':   4,
 '002':   5,
 '003':   6,
 '004':   7,
 '005':   8,
 '006':   9,
 ...
}
index of UNK: 332639


train_text_X의 단어들의 정렬 순서에 해당하는 값을 text_as_int에 저장한다.

In [ ]:
print(train_text_X[:20])
print(text_as_int[:20])

['태조', '이성계', '선대의', '가계', '목조', '이안사가', '전주에서', '삼척', '의주를', '거쳐', '알동에', '정착하다', '\n', '태조', '강헌', '지인', '계운', '성문', '신무', '대왕']
[299305 229634 161443  17430 111029 230292 251081 155087 225462  29027
 190295 256129      0 299305  25624 273553  36147 163996 180466  84413]


25 단어가 주어지면 다음 단어를 예측하도록 신경망을 학습시키고자 한다. seq_length에는 25를 저장하고, examples_per_epoch에는 전체 text의 길이를 seq_length로 나눈 몫을 저장한다.

sentence_dataset은 text_as_int의 각 단어들을 Dataset 형태로 저장한 것이다. 이를 seq_length(25 입력 + 1 출력)만큼씩 나누어 저장한다. 나머지는 drop_remainder 옵션으로 버릴 수 있다.

dataset에 대한 자세한 내용은
https://www.tensorflow.org/api_docs/python/tf/data/Dataset
참고


In [ ]:
seq_length = 25
# seq_length가 25이면 25개의 단어가 주어졌을 때 다음 단어를 예측하도록 할 수 있음.
examples_per_epoch = len(text_as_int) // seq_length

sentence_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sentence_dataset = sentence_dataset.batch(seq_length+1, drop_remainder=True)
# batch()함수의 인자로는 반환되는 데이터의 숫자를 지정함. 
# 첫 25단어는 입력이고 마지막 한 단어는 정답.
# drop_remainder=True로 남는 부분은 버림.

for item in sentence_dataset.take(1):
    print(idx2word[item.numpy()])
    print(item.numpy())

['태조' '이성계' '선대의' '가계' '목조' '이안사가' '전주에서' '삼척' '의주를' '거쳐' '알동에' '정착하다'
 '\n' '태조' '강헌' '지인' '계운' '성문' '신무' '대왕' '의' '성은' '이씨' '요' ',' '휘']
[299305 229634 161443  17430 111029 230292 251081 155087 225462  29027
 190295 256129      0 299305  25624 273553  36147 163996 180466  84413
 224182 164549 230248 210912      2 330313]


take() : https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take

from_tensor_slices() : https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices


26개의 단어를 다시 25개의 입력, 1개의 정답으로 나누어준다.

In [ ]:
def split_input_target(chunk):
    return [chunk[:-1], chunk[-1]]
# 26개의 단어를 25개, 1개로 잘라주는 함수.

train_dataset = sentence_dataset.map(split_input_target)

In [ ]:
# 25개의 입력과 정답을 출력함(3번)

for x, y in train_dataset.take(3):
    print(idx2word[x.numpy()])
    print(x.numpy())
    print(idx2word[y.numpy()])
    print(y.numpy())

['태조' '이성계' '선대의' '가계' '목조' '이안사가' '전주에서' '삼척' '의주를' '거쳐' '알동에' '정착하다'
 '\n' '태조' '강헌' '지인' '계운' '성문' '신무' '대왕' '의' '성은' '이씨' '요' ',']
[299305 229634 161443  17430 111029 230292 251081 155087 225462  29027
 190295 256129      0 299305  25624 273553  36147 163996 180466  84413
 224182 164549 230248 210912      2]
휘
330313
['는' '단' '이요' ',' '자' '는' '군진' '이다' '그전의' '휘' '는' '이성계' '요' ',' '호' '는'
 '송헌' '이다' '전주' '의' '대성' '이다' '사공' '휘' '이한']
[ 77850  80105 230992      2 240316  77850  49621 226910  53932 330313
  77850 229634 210912      2 322525  77850 170794 226910 251070 224182
  83973 226910 147977 330313 233123]
이
225845
['신라' '에' '벼슬하여' '태종왕' '001' '의' '10대' '손자인' '군윤' '김은의' '의' '딸에게' '장가들어'
 '시중' '휘' '이자연' '을' '낳았다' '시중이' '복야' '휘' '이천상' '을' '낳고' ',']
[180269 197963 131090 299389      4 224182    359 169777  49369  62515
 224182  98407 244180 178763 330313 231753 223046  70716 178782 136380
 330313 232680 223046  70689      2]
복야가
136381


In [ ]:
BATCH_SIZE = 512
steps_per_epoch = examples_per_epoch // BATCH_SIZE
BUFFER_SIZE = 10000

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
total_words = len(vocab)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=seq_length),
    tf.keras.layers.LSTM(units=100, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units=100),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 100)           33264000  
_________________________________________________________________
lstm (LSTM)                  (None, 25, 100)           80400     
_________________________________________________________________
dropout (Dropout)            (None, 25, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 332640)            33596640  
Total params: 67,021,440
Trainable params: 67,021,440
Non-trainable params: 0
_________________________________________________________________


testmodel 함수는 5번째 학습마다 결과를 출력한다. 매 시도마다 train_text[0]("태조 이성계 선대의 가계 목조 이안사가 전주에서 삼척 의주를 거쳐 알동에 정착하다") 이후의 100단어를 출력한다.

predict_classes는 test_sentence 다음에 올 단어의 인덱스를 반환하고, idx2word 리스트를 통해 다음 단어를 test_sentence에 연결한다.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def testmodel(epoch, logs):
    if epoch % 5 != 0 and epoch != 49:
        return
    test_sentence = train_text[0]

    next_words = 100
    for _ in range(next_words):
        test_text_X = test_sentence.split(' ')[-seq_length:]
        test_text_X = np.array([word2idx[c] if c in word2idx else word2idx['UNK'] for c in test_text_X])
        test_text_X = pad_sequences([test_text_X], maxlen=seq_length, padding='pre', value=word2idx['UNK'])

        output_idx = model.predict_classes(test_text_X)
        test_sentence += ' ' + idx2word[output_idx[0]]
    
    print()
    print(test_sentence)
    print()

testmodelcb = tf.keras.callbacks.LambdaCallback(on_epoch_end=testmodel)

history = model.fit(train_dataset.repeat(), epochs=50, steps_per_epoch=steps_per_epoch, callbacks=[testmodelcb], verbose=2)

Epoch 1/50
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).

 태조 이성계 선대의 가계 목조 이안사가 전주에서 삼척 의주를 거쳐 알동에 정착하다  , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,

533/533 - 411s - loss: 9.3794 - accuracy: 0.0721
Epoch 2/50
533/533 - 405s - loss: 8.3608 - accuracy: 0.0740
Epoch 3/50
533/533 - 404s - loss: 8.0752 - accuracy: 0.0812
Epoch 4/50
533/533 - 404s - loss: 7.8220 - accuracy: 0.0897
Epoch 5/50
533/533 - 403s - loss: 7.5649 - accuracy: 0.1001
Epoch 6/50

 태조 이성계 선대의 가계 목조 이안사가 전주에서 삼척 의주를 거쳐 알동에 정착하다  것을 보내어 , , 그 말하기를 , 
 임금이 , 
 , 
 하니 , 
 하니 , 
 하니 , 
 하

test_sentence에 난중일기의 일부분을 주고, 이후 100단어를 출력하도록 한다.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

test_sentence = '동헌에 나가 공무를 본 후 활 십오 순을 쏘았다'

next_words = 100
for _ in range(next_words):
    test_text_X = test_sentence.split(' ')[-seq_length:]
    test_text_X = np.array([word2idx[c] if c in word2idx else word2idx['UNK'] for c in test_text_X])
    test_text_X = pad_sequences([test_text_X], maxlen=seq_length, padding='pre', value=word2idx['UNK'])
    
    output_idx = model.predict_classes(test_text_X)
    test_sentence += ' ' + idx2word[output_idx[0]]

print(test_sentence)

동헌에 나가 공무를 본 후 활 십오 순을 쏘았다 간격이 대하는 하되 , 약재 는 군 도회소 에 내보이면서 석 인 제읍 인 더불어 영속 이극감 김계손 김계손 행 신종윤 115 밀성군 039 들에게 기르면서 등과 진 정창손 김계손 행 김계손 병조 중추원 이극배 의창 을 이관 겸 의창 579 를 점검하여 채우고 \? 
 윤 에 이르렀고 , 대신 에 일일이 베풀고 , 인하여 혹은 상 을 바쳤다 진무 를 변별하는 , 매양 조례의 를 쌓아 상 를 갖추고 1년에 상 를 지키고 하여 항상 실정 에 죽었다 정치를 한다면 한갓 죽음을 군사들 없었던 일입니다 그러나 어찌하여 언관 등으로 율 에 속하게 합니다 그리고 정인지 정식 좌부승지 박강
